In [1]:
import os
os.chdir("D:\\UIT\\HK\\HK6\\IS403\\Đồ án\\Source code")
print("Thư mục làm việc hiện tại:", os.getcwd())

Thư mục làm việc hiện tại: D:\UIT\HK\HK6\IS403\Đồ án\Source code


# Import modules

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import joblib

In [5]:
import json

In [6]:
import time

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [ ]:
from model_application.evaluation_errors_utils import error

# Load and preprocess data

In [9]:
def load_and_prepare_data(path):
    dataframe = pd.read_csv(path)
    dataframe["Time"] = pd.to_datetime(dataframe["Time"])

    dataframe["Close_lag1"] = dataframe["Close"].shift(1)
    dataframe["MA5"] = dataframe["Close"].rolling(5).mean()
    dataframe["Return_1day"] = dataframe["Close"].pct_change(1)

    dataframe = dataframe.dropna()
    X = dataframe.drop(columns=["Close", "Time"])
    y = dataframe[["Close"]]
    
    return X, y


def prepare_multi_step_data(X, y, n_steps=4):
    y_multi = pd.DataFrame()
    for i in range(n_steps):
        y_multi[f"step_{i+1}"] = y.shift(-i)

    valid_indices = ~y_multi.isna().any(axis=1)
    
    X = X[valid_indices]
    y_multi = y_multi[valid_indices]

    return X, y_multi


def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

In [10]:
N_STEPS = 4

## Vietcombank

In [11]:
vcb_X_train, vcb_y_train = load_and_prepare_data(".\\dataset\\official_dataset\\vcb\\vcb_train.csv")
vcb_X_validation, vcb_y_validation = load_and_prepare_data(".\\dataset\\official_dataset\\vcb\\vcb_validation.csv")
vcb_X_test, vcb_y_test = load_and_prepare_data(".\\dataset\\official_dataset\\vcb\\vcb_test.csv")

In [12]:
vcb_X_train = pd.concat([vcb_X_train, vcb_X_validation])
vcb_y_train = pd.concat([vcb_y_train, vcb_y_validation])

In [13]:
vcb_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25013.62,595.78,22540.0,0.153907,382860590593900,23038.541622,111170655000000,45.20,44.790,-0.003761
5,1467490,25033.58,593.93,22510.0,0.153907,382860590593900,23038.541622,111170655000000,45.03,44.836,-0.001332
6,1467490,25064.52,597.29,22530.0,0.153907,382860590593900,23038.541622,111170655000000,44.97,45.062,0.007561
7,1467490,25148.56,602.50,22530.0,0.153907,382860590593900,23038.541622,111170655000000,45.31,45.378,0.023615
8,1467490,25303.38,602.68,22530.0,0.153907,382860590593900,23038.541622,111170655000000,46.38,45.784,0.018327
...,...,...,...,...,...,...,...,...,...,...,...
80,1561963,25792.47,657.38,25193.0,0.178524,512519668725400,34028.114605,190297001000000,61.61,61.782,0.003246
81,1561963,25779.62,658.40,25206.0,0.178524,512519668725400,34028.114605,190297001000000,61.81,61.742,-0.003236
82,1561963,25747.18,657.18,25195.0,0.178524,512519668725400,34028.114605,190297001000000,61.61,61.702,0.000974
83,1561963,25817.03,658.27,25208.0,0.178524,512519668725400,34028.114605,190297001000000,61.67,61.648,-0.002108


In [14]:
vcb_y_train

,Close
4,45.03
5,44.97
6,45.31
7,46.38
8,47.23
...,...
80,61.81
81,61.61
82,61.67
83,61.54


In [15]:
vcb_X_train, vcb_y_train = prepare_multi_step_data(vcb_X_train, vcb_y_train, N_STEPS)
vcb_X_test, vcb_y_test = prepare_multi_step_data(vcb_X_test, vcb_y_test, N_STEPS)

In [16]:
vcb_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25013.62,595.78,22540.0,0.153907,382860590593900,23038.541622,111170655000000,45.20,44.790,-0.003761
5,1467490,25033.58,593.93,22510.0,0.153907,382860590593900,23038.541622,111170655000000,45.03,44.836,-0.001332
6,1467490,25064.52,597.29,22530.0,0.153907,382860590593900,23038.541622,111170655000000,44.97,45.062,0.007561
7,1467490,25148.56,602.50,22530.0,0.153907,382860590593900,23038.541622,111170655000000,45.31,45.378,0.023615
8,1467490,25303.38,602.68,22530.0,0.153907,382860590593900,23038.541622,111170655000000,46.38,45.784,0.018327
...,...,...,...,...,...,...,...,...,...,...,...
77,1561963,25725.51,651.10,25189.0,0.178524,512519668725400,34028.114605,190297001000000,61.81,62.086,0.000971
78,1561963,25703.29,651.79,25210.0,0.178524,512519668725400,34028.114605,190297001000000,61.87,61.994,-0.000970
79,1561963,25864.38,656.23,25200.0,0.178524,512519668725400,34028.114605,190297001000000,61.81,61.874,-0.003236
80,1561963,25792.47,657.38,25193.0,0.178524,512519668725400,34028.114605,190297001000000,61.61,61.782,0.003246


In [17]:
vcb_y_train

,step_1,step_2,step_3,step_4
4,45.03,44.97,45.31,46.38
5,44.97,45.31,46.38,47.23
6,45.31,46.38,47.23,48.82
7,46.38,47.23,48.82,49.61
8,47.23,48.82,49.61,49.16
...,...,...,...,...
77,61.87,61.81,61.61,61.81
78,61.81,61.61,61.81,61.61
79,61.61,61.81,61.61,61.67
80,61.81,61.61,61.67,61.54


In [18]:
vcb_X_train_scaled, vcb_X_test_scaled = scale_data(vcb_X_train,vcb_X_test)

In [19]:
vcb_X_train_scaled

array([[ 0.17248373, -0.23331184, -0.62266174, ..., -1.24742219,
        -1.3042744 , -0.30341451],
       [ 0.17248373, -0.21602718, -0.69152376, ..., -1.27045868,
        -1.29802008, -0.13000102],
       [ 0.17248373, -0.18923424, -0.56645544, ..., -1.2785892 ,
        -1.26729235,  0.50499962],
       ...,
       [ 1.06091341,  0.5034161 ,  1.62745131, ...,  1.00337788,
         1.01852432, -0.26590304],
       [ 1.06091341,  0.44114459,  1.67025744, ...,  0.97627614,
         1.00601569,  0.19693543],
       [ 1.06091341,  0.43001695,  1.7082246 , ...,  1.00337788,
         1.00057715, -0.26590304]])

## BIDV

In [20]:
bid_X_train, bid_y_train = load_and_prepare_data(".\\dataset\\official_dataset\\bid\\bid_train.csv")
bid_X_validation, bid_y_validation = load_and_prepare_data(".\\dataset\\official_dataset\\bid\\bid_validation.csv")
bid_X_test, bid_y_test = load_and_prepare_data(".\\dataset\\official_dataset\\bid\\bid_test.csv")

In [21]:
bid_X_train = pd.concat([bid_X_train, bid_X_validation])
bid_y_train = pd.concat([bid_y_train, bid_y_validation])

In [22]:
bid_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25209.0,636.69,22495.0,0.355021,188430012146000,16434.655845,86366645000000,28.75,28.022,0.004870
5,1467490,25270.0,640.67,22535.0,0.355021,188430012146000,16434.655845,86366645000000,28.89,28.718,0.069920
6,1467490,25328.0,643.55,22515.0,0.355021,188430012146000,16434.655845,86366645000000,30.91,29.738,0.043999
7,1467490,25481.0,646.85,22515.0,0.355021,188430012146000,16434.655845,86366645000000,32.27,30.654,0.005578
8,1467490,25573.0,647.33,22525.0,0.355021,188430012146000,16434.655845,86366645000000,32.45,31.306,-0.013559
...,...,...,...,...,...,...,...,...,...,...,...
75,1561963,25981.0,684.02,24219.0,0.124035,269003095452000,19333.031073,138439656000000,38.10,38.426,0.014436
76,1561963,26014.0,683.64,24227.0,0.124035,269003095452000,19333.031073,138439656000000,38.65,38.548,0.009056
77,1561963,25983.0,682.94,24216.0,0.124035,269003095452000,19333.031073,138439656000000,39.00,38.700,0.003846
78,1561963,26018.0,683.55,24209.0,0.124035,269003095452000,19333.031073,138439656000000,39.15,38.630,-0.022989


In [23]:
bid_y_train

,Close
4,28.89
5,30.91
6,32.27
7,32.45
8,32.01
...,...
75,38.65
76,39.00
77,39.15
78,38.25


In [24]:
bid_X_train, bid_y_train = prepare_multi_step_data(bid_X_train, bid_y_train, N_STEPS)
bid_X_test, bid_y_test = prepare_multi_step_data(bid_X_test, bid_y_test, N_STEPS)

In [25]:
bid_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25209.0,636.69,22495.0,0.355021,188430012146000,16434.655845,86366645000000,28.75,28.022,0.004870
5,1467490,25270.0,640.67,22535.0,0.355021,188430012146000,16434.655845,86366645000000,28.89,28.718,0.069920
6,1467490,25328.0,643.55,22515.0,0.355021,188430012146000,16434.655845,86366645000000,30.91,29.738,0.043999
7,1467490,25481.0,646.85,22515.0,0.355021,188430012146000,16434.655845,86366645000000,32.27,30.654,0.005578
8,1467490,25573.0,647.33,22525.0,0.355021,188430012146000,16434.655845,86366645000000,32.45,31.306,-0.013559
...,...,...,...,...,...,...,...,...,...,...,...
72,1561963,25982.0,674.70,24210.0,0.124035,269003095452000,19333.031073,138439656000000,38.39,38.528,0.000000
73,1561963,25996.0,677.41,24230.0,0.124035,269003095452000,19333.031073,138439656000000,38.39,38.522,0.005470
74,1561963,26078.0,682.88,24221.0,0.124035,269003095452000,19333.031073,138439656000000,38.60,38.440,-0.012953
75,1561963,25981.0,684.02,24219.0,0.124035,269003095452000,19333.031073,138439656000000,38.10,38.426,0.014436


In [26]:
bid_y_train

,step_1,step_2,step_3,step_4
4,28.89,30.91,32.27,32.45
5,30.91,32.27,32.45,32.01
6,32.27,32.45,32.01,32.93
7,32.45,32.01,32.93,32.01
8,32.01,32.93,32.01,34.25
...,...,...,...,...
72,38.39,38.60,38.10,38.65
73,38.60,38.10,38.65,39.00
74,38.10,38.65,39.00,39.15
75,38.65,39.00,39.15,38.25


In [27]:
bid_X_train_scaled, bid_X_test_scaled = scale_data(bid_X_train, bid_X_test)

In [28]:
bid_X_train_scaled

array([[ 0.17204362, -0.22995476, -0.72841088, ..., -0.977977  ,
        -1.11158533,  0.21892105],
       [ 0.17204362, -0.1778804 , -0.55349184, ..., -0.95266408,
        -0.98526923,  3.48635246],
       [ 0.17204362, -0.12836708, -0.42691726, ..., -0.58743484,
        -0.80015079,  2.18433484],
       ...,
       [ 1.06108917,  0.51189137,  1.30161689, ...,  0.80296758,
         0.7791635 , -0.67630532],
       [ 1.06108917,  0.42908461,  1.35171933, ...,  0.7125643 ,
         0.77662266,  0.69941728],
       [ 1.06108917,  0.45725598,  1.33501851, ...,  0.81200791,
         0.79876428,  0.42918236]])

## Techcombank

In [29]:
tcb_X_train, tcb_y_train = load_and_prepare_data(".\\dataset\\official_dataset\\tcb\\tcb_train.csv")
tcb_X_validation, tcb_y_validation = load_and_prepare_data(".\\dataset\\official_dataset\\tcb\\tcb_validation.csv")
tcb_X_test, tcb_y_test = load_and_prepare_data(".\\dataset\\official_dataset\\tcb\\tcb_test.csv")

In [30]:
tcb_X_train = pd.concat([tcb_X_train, tcb_X_validation])
tcb_y_train = pd.concat([tcb_y_train, tcb_y_validation])

In [31]:
tcb_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25188.0,596.0,22521.0,0.161678,160097714788800,26259.416507,93055837000000,24.44,24.178,0.002046
5,1467490,25432.0,599.0,22535.0,0.161678,160097714788800,26259.416507,93055837000000,24.49,24.104,-0.015108
6,1467490,25371.0,604.0,22557.0,0.161678,160097714788800,26259.416507,93055837000000,24.12,24.036,-0.019900
7,1467490,25308.0,605.0,22574.0,0.161678,160097714788800,26259.416507,93055837000000,23.64,24.076,0.002115
8,1467490,25122.0,601.0,22525.0,0.161678,160097714788800,26259.416507,93055837000000,23.69,23.926,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
62,1561963,25864.0,660.0,25225.0,0.243784,166730501040400,20259.284053,144367982000000,23.80,23.990,0.008403
63,1561963,25810.0,660.0,25210.0,0.243784,166730501040400,20259.284053,144367982000000,24.00,24.040,0.018750
64,1561963,25848.0,659.0,25205.0,0.243784,166730501040400,20259.284053,144367982000000,24.45,24.070,-0.004090
65,1561963,25828.0,658.0,25210.0,0.243784,166730501040400,20259.284053,144367982000000,24.35,24.240,0.010267


In [32]:
tcb_y_train

,Close
4,24.49
5,24.12
6,23.64
7,23.69
8,23.69
...,...
62,24.00
63,24.45
64,24.35
65,24.60


In [33]:
tcb_X_train, tcb_y_train = prepare_multi_step_data(tcb_X_train, tcb_y_train, N_STEPS)
tcb_X_test, tcb_y_test = prepare_multi_step_data(tcb_X_test, tcb_y_test, N_STEPS)

In [34]:
tcb_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25188.0,596.0,22521.0,0.161678,160097714788800,26259.416507,93055837000000,24.44,24.178,0.002046
5,1467490,25432.0,599.0,22535.0,0.161678,160097714788800,26259.416507,93055837000000,24.49,24.104,-0.015108
6,1467490,25371.0,604.0,22557.0,0.161678,160097714788800,26259.416507,93055837000000,24.12,24.036,-0.019900
7,1467490,25308.0,605.0,22574.0,0.161678,160097714788800,26259.416507,93055837000000,23.64,24.076,0.002115
8,1467490,25122.0,601.0,22525.0,0.161678,160097714788800,26259.416507,93055837000000,23.69,23.926,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
59,1561963,26056.0,657.0,25210.0,0.243784,166730501040400,20259.284053,144367982000000,24.20,24.230,0.000000
60,1561963,26090.0,659.0,25233.0,0.243784,166730501040400,20259.284053,144367982000000,24.20,24.150,-0.018595
61,1561963,25879.0,653.0,25240.0,0.243784,166730501040400,20259.284053,144367982000000,23.75,24.060,0.002105
62,1561963,25864.0,660.0,25225.0,0.243784,166730501040400,20259.284053,144367982000000,23.80,23.990,0.008403


In [35]:
tcb_y_train

,step_1,step_2,step_3,step_4
4,24.49,24.12,23.64,23.69
5,24.12,23.64,23.69,23.69
6,23.64,23.69,23.69,24.15
7,23.69,23.69,24.15,25.17
8,23.69,24.15,25.17,25.41
...,...,...,...,...
59,24.20,23.75,23.80,24.00
60,23.75,23.80,24.00,24.45
61,23.80,24.00,24.45,24.35
62,24.00,24.45,24.35,24.60


In [36]:
tcb_X_train_scaled, tcb_X_test_scaled = scale_data(tcb_X_train, tcb_X_test)

In [37]:
tcb_X_train_scaled

array([[ 0.18111181, -0.12919559, -0.43670702, ...,  1.27148647,
         1.21645901,  0.07456274],
       [ 0.18111181,  0.07731465, -0.3370172 , ...,  1.28304395,
         1.19928556, -0.70783461],
       [ 0.18111181,  0.02568709, -0.17086752, ...,  1.19751862,
         1.18350456, -0.92641148],
       ...,
       [ 1.06514016,  0.45563463,  1.45739941, ...,  1.1119933 ,
         1.18907432,  0.07727365],
       [ 1.06514016,  0.44293933,  1.69000897, ...,  1.12355078,
         1.17282917,  0.36453059],
       [ 1.06514016,  0.39723624,  1.69000897, ...,  1.16978068,
         1.18443285,  0.8364419 ]])

# Excute model

In [38]:
models = {
    "random_forest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10]
        },
        "use_scaled": False,
        "weight": 0.8
    },
    "svm": {
        "model": SVR(),
        "params": {
            "kernel": ["linear", "rbf"],
            "C": [0.1, 1, 10],
            "epsilon": [0.01, 0.1]
        },
        "use_scaled": True,
        "weight": 0.7
    },
    "linear_regression": {
        "model": LinearRegression(),
        "params": {},
        "use_scaled": True,
        "weight": 0.6
    },
    "ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [0.1, 1.0, 10.0]
        },
        "use_scaled": True,
        "weight": 0.5
    }
}

In [39]:
def find_best_models(X_train, X_train_scaled, y_train, n_steps, checkpoint_path):
    best_models = {}
    for name, config in models.items():
        start_time = time.time()
        print(f"\nĐang tối ưu mô hình {name}...")
        X_tr = X_train_scaled if config["use_scaled"] else X_train

        if config["params"]:
            tscv = TimeSeriesSplit(n_splits=5)
            gscv = GridSearchCV(
                estimator=config["model"],
                param_grid=config["params"],
                cv=tscv,
                scoring="neg_mean_squared_error",
                n_jobs=-1,
                verbose=0
            )
            step_models = []
            for step in range(n_steps):
                print(f"Step {step + 1}/{n_steps}")
                gscv.fit(X_tr, y_train.iloc[:, step])
                step_models.append(gscv.best_estimator_)
                print(f"Best params: {gscv.best_params_}")
            best_models[name] = step_models
        else:
            step_models = []
            for step in range(n_steps):
                print(f"Step {step + 1}/{n_steps}")
                model = config["model"]
                model.fit(X_tr, y_train.iloc[:, step])
                step_models.append(model)
            best_models[name] = step_models
        end_time = time.time()

        training_time = end_time - start_time
        json.dump(
            {"training time": training_time},
            open(os.path.join(checkpoint_path, f"{name}_training_time.json"), "w+"),
            ensure_ascii=False,
            indent=4
        )

    for name, step_models in best_models.items():
        file_path = os.path.join(checkpoint_path, f"{name}.pkl")
        with open(file_path, "wb") as file:
            joblib.dump(step_models, file)
        print(f"Đã lưu mô hình {name} vào file: {file_path}")
    

def evaluate(X_test, X_test_scaled, y_test, checkpoint_path, n_steps):
    for name, config in models.items():
        X_t = X_test_scaled if config["use_scaled"] else X_test
        model = joblib.load(os.path.join(checkpoint_path, f"{name}.pkl"))
        start_time = time.time()
        for step in range(n_steps):
            prediction = model[step].predict(X_t)
            prediction = np.expand_dims(prediction, axis=1)
            if step == 0:
                predictions = prediction
            else:
                predictions = np.hstack((predictions, prediction))

        end_time = time.time()

        mse, rmse, mae, mape = error(predictions, y_test)
        elapsed = end_time - start_time

        json.dump(
            {
                "time": elapsed,
                "predictions": predictions.tolist(),
                "targets": y_test.values.tolist(),
                "mse": mse,
                "rmse": rmse,
                "mae": mae,
                "mape": mape
            },
            open(os.path.join(checkpoint_path, f"{name}_test_results.json"), "w+"),
            ensure_ascii=False,
            indent=4
        )


def get_predictions(features, checkpoint_path, model_name, n_steps):
    model = joblib.load(os.path.join(checkpoint_path, f"{model_name}.pkl"))
    predictions = []
    for step in range(n_steps):
        prediction = model[step].predict(features)
        predictions.append(prediction)

    json.dump(
        {"predictions": predictions.tolist()},
        open(os.path.join(checkpoint_path, "predictions.json"), "w+"),
        ensure_ascii=False,
        indent=4
    )

## Vietcombank

In [40]:
vcb_checkpoint_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\multi_steps\\vcb"

In [41]:
find_best_models(vcb_X_train, vcb_X_train_scaled, vcb_y_train, N_STEPS, vcb_checkpoint_path)


Đang tối ưu mô hình random_forest...
Step 1/4
Best params: {'max_depth': None, 'n_estimators': 100}
Step 2/4
Best params: {'max_depth': 10, 'n_estimators': 100}
Step 3/4
Best params: {'max_depth': None, 'n_estimators': 100}
Step 4/4
Best params: {'max_depth': 10, 'n_estimators': 200}

Đang tối ưu mô hình svm...
Step 1/4
Best params: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}
Step 2/4
Best params: {'C': 1, 'epsilon': 0.01, 'kernel': 'linear'}
Step 3/4
Best params: {'C': 1, 'epsilon': 0.1, 'kernel': 'linear'}
Step 4/4
Best params: {'C': 1, 'epsilon': 0.01, 'kernel': 'linear'}

Đang tối ưu mô hình linear_regression...
Step 1/4
Step 2/4
Step 3/4
Step 4/4

Đang tối ưu mô hình ridge...
Step 1/4
Best params: {'alpha': 0.1}
Step 2/4
Best params: {'alpha': 0.1}
Step 3/4
Best params: {'alpha': 1.0}
Step 4/4
Best params: {'alpha': 1.0}
Đã lưu mô hình random_forest vào file: .\model_application\experiment\saved_checkpoint\machine_learning\multi_steps\vcb\random_forest.pkl
Đã lưu mô hình svm v

In [42]:
evaluate(vcb_X_test, vcb_X_test_scaled, vcb_y_test, vcb_checkpoint_path, N_STEPS)

## BIDV

In [43]:
bid_checkpoint_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\multi_steps\\bid"

In [44]:
find_best_models(bid_X_train, bid_X_train_scaled, bid_y_train, N_STEPS, bid_checkpoint_path)


Đang tối ưu mô hình random_forest...
Step 1/4
Best params: {'max_depth': None, 'n_estimators': 200}
Step 2/4
Best params: {'max_depth': 10, 'n_estimators': 100}
Step 3/4
Best params: {'max_depth': 10, 'n_estimators': 200}
Step 4/4
Best params: {'max_depth': None, 'n_estimators': 200}

Đang tối ưu mô hình svm...
Step 1/4
Best params: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}
Step 2/4
Best params: {'C': 1, 'epsilon': 0.01, 'kernel': 'linear'}
Step 3/4
Best params: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'linear'}
Step 4/4
Best params: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'linear'}

Đang tối ưu mô hình linear_regression...
Step 1/4
Step 2/4
Step 3/4
Step 4/4

Đang tối ưu mô hình ridge...
Step 1/4
Best params: {'alpha': 0.1}
Step 2/4
Best params: {'alpha': 1.0}
Step 3/4
Best params: {'alpha': 10.0}
Step 4/4
Best params: {'alpha': 10.0}
Đã lưu mô hình random_forest vào file: .\model_application\experiment\saved_checkpoint\machine_learning\multi_steps\bid\random_forest.pkl
Đã lưu mô hình

In [45]:
evaluate(bid_X_test, bid_X_test_scaled, bid_y_test, bid_checkpoint_path, N_STEPS)

## Techcombank

In [46]:
tcb_checkpoint_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\multi_steps\\tcb"

In [47]:
find_best_models(tcb_X_train, tcb_X_train_scaled, tcb_y_train, N_STEPS, tcb_checkpoint_path)


Đang tối ưu mô hình random_forest...
Step 1/4
Best params: {'max_depth': None, 'n_estimators': 100}
Step 2/4
Best params: {'max_depth': 10, 'n_estimators': 100}
Step 3/4
Best params: {'max_depth': None, 'n_estimators': 100}
Step 4/4
Best params: {'max_depth': 10, 'n_estimators': 200}

Đang tối ưu mô hình svm...
Step 1/4
Best params: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}
Step 2/4
Best params: {'C': 1, 'epsilon': 0.1, 'kernel': 'linear'}
Step 3/4
Best params: {'C': 1, 'epsilon': 0.1, 'kernel': 'linear'}
Step 4/4
Best params: {'C': 1, 'epsilon': 0.1, 'kernel': 'linear'}

Đang tối ưu mô hình linear_regression...
Step 1/4
Step 2/4
Step 3/4
Step 4/4

Đang tối ưu mô hình ridge...
Step 1/4
Best params: {'alpha': 0.1}
Step 2/4
Best params: {'alpha': 0.1}
Step 3/4
Best params: {'alpha': 1.0}
Step 4/4
Best params: {'alpha': 1.0}
Đã lưu mô hình random_forest vào file: .\model_application\experiment\saved_checkpoint\machine_learning\multi_steps\tcb\random_forest.pkl
Đã lưu mô hình svm vào 

In [48]:
evaluate(tcb_X_test, tcb_X_test_scaled, tcb_y_test, tcb_checkpoint_path, N_STEPS)